In [1]:
import pandas as pd, numpy as np
import sys, os.path
sys.path.append(os.path.abspath("../.."))
from pre_processing.id_helper import *
import rank_countries_by_stunting as rbs
import pull_gbd2019_data as pgd

from db_queries import get_ids, get_population, get_outputs, get_best_model_versions
from get_draws.api import get_draws

!date
!whoami

Tue Jan 19 12:14:28 PST 2021
ndbs


In [2]:
%load_ext autoreload
%autoreload 2

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Test going back and forth between a MultiIndex and a flat index

Looks like it works!

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.from_tuples.html

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.to_flat_index.html

In [3]:
tuples = [(1, 'red'), (1, 'blue'),
          (2, 'red'), (2, 'blue')]
idx = pd.MultiIndex.from_tuples(tuples, names=('number', 'color'))
idx

MultiIndex([(1,  'red'),
            (1, 'blue'),
            (2,  'red'),
            (2, 'blue')],
           names=['number', 'color'])

In [4]:
idx.to_flat_index()

Index([(1, 'red'), (1, 'blue'), (2, 'red'), (2, 'blue')], dtype='object')

In [6]:
pd.MultiIndex.from_tuples(idx.to_flat_index(), names=idx.names)

MultiIndex([(1,  'red'),
            (1, 'blue'),
            (2,  'red'),
            (2, 'blue')],
           names=['number', 'color'])

# Define locations

In [7]:
locations = pgd.get_locations('top25')
locations = pgd.get_or_append_global_location(locations)
locations

,location_name,location_id
0,Angola,168
1,Bangladesh,161
2,Burkina Faso,201
3,Cameroon,202
4,China,6
5,Côte d'Ivoire,205
6,Democratic Republic of the Congo,171
7,Egypt,141
8,Ethiopia,179
9,Ghana,207


# Load zinc data

In [8]:
!ls data

all_countries_with_ids.csv
bgmf_countries.csv
bgmf_countries_with_ids.csv
bmgf_top_25_countries_20201203.csv
countries_ranked_by_number_stunted_2019.csv
countries_with_over_20_percent_stunting_ranked_by_number_stunted_2019.csv
lsff_gbd_data_summary.hdf
neural_tube_defects_output_gbd2017.csv
percent_global_burden_mn_deficiency_for_top25.csv
stunting_prevalence_2019.hdf
top_25_countries_with_stunting_above_18_percent_in_2019.csv
top_25_countries_with_stunting_above_20_percent_in_2019.csv
vitamin_a_data_summary.csv


/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:
hdf_filepath='data/lsff_gbd_data_summary.hdf'

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
zinc_prevalence = pd.read_hdf('data/lsff_gbd_data_summary.hdf',
                              'zinc_deficiency/prevalence_among_1_to_4')
zinc_population = pd.read_hdf('data/lsff_gbd_data_summary.hdf',
                              'zinc_deficiency/population_1_to_4')
zinc_burden = pd.read_hdf('data/lsff_gbd_data_summary.hdf',
                         'zinc_deficiency/daly_burden')

In [11]:
zinc_preprocessed = pgd.preprocess_zinc_data((zinc_prevalence, zinc_population, zinc_burden))
len(zinc_preprocessed)

3

In [51]:
zinc_prevalence.head()

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,parameter,measure_id,metric_id,draw_0,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
0,97,2544.0,1,2019,5,1,cat1,18,3,0.079500,...,0.096346,0.132976,0.060709,0.071232,0.128059,0.139341,0.087399,0.067168,0.080435,0.130299
1,97,2544.0,1,2019,5,2,cat1,18,3,0.077895,...,0.093071,0.127964,0.053030,0.065427,0.124007,0.133193,0.086742,0.059063,0.075436,0.126039
2,97,2544.0,11,2019,5,1,cat1,18,3,0.068182,...,0.085701,0.139703,0.046390,0.063405,0.132635,0.142091,0.080776,0.059324,0.072853,0.131697
3,97,2544.0,11,2019,5,2,cat1,18,3,0.068561,...,0.085989,0.137388,0.036832,0.050506,0.110897,0.146077,0.072973,0.045500,0.065609,0.129610
4,97,2544.0,141,2019,5,1,cat1,18,3,0.021719,...,0.029482,0.054900,0.006386,0.015178,0.049381,0.060945,0.025866,0.012109,0.020428,0.056824


In [52]:
zinc_population.head()

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,age_group_id,location_id,year_id,sex_id,population,upper,lower,run_id
0,5,1,2019,1,2.741937e+08,2.822446e+08,2.662198e+08,192
1,5,1,2019,2,2.566635e+08,2.639914e+08,2.493610e+08,192
2,5,1,2019,3,5.308572e+08,5.462795e+08,5.156214e+08,192
3,5,6,2019,1,3.575678e+07,4.016035e+07,3.114844e+07,192
4,5,6,2019,2,3.066608e+07,3.444171e+07,2.671444e+07,192


In [53]:
zinc_burden.head()

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,metric_id,rei_id,sex_id,year_id
0,5,294,127295.614263,45664.775930,63403.650313,116720.947920,35218.124496,30873.467379,40736.543242,128643.398992,...,91531.825555,74724.512286,79480.297639,101998.204573,1,2,1,97,2,2019
1,5,294,125464.769419,52515.863706,68302.347137,134367.755366,38005.300731,32689.250201,44763.301712,129657.854605,...,104645.816809,71854.306497,82522.959922,116909.205102,1,2,1,97,1,2019
2,5,294,0.001749,0.000769,0.001035,0.001643,0.000574,0.000510,0.000722,0.001875,...,0.001741,0.001066,0.001321,0.001885,1,2,2,97,1,2019
3,5,294,0.001906,0.000718,0.001036,0.001593,0.000571,0.000518,0.000693,0.002000,...,0.001635,0.001197,0.001371,0.001781,1,2,2,97,2,2019
4,5,294,954.532812,197.702087,436.339271,1209.191167,371.705455,208.350528,464.778852,1270.639991,...,1282.128241,957.580283,1063.576027,1460.532078,11,2,1,97,2,2019


# Make tables

In [31]:
zinc_summary = pgd.get_binary_risk_data_summary(zinc_preprocessed)
zinc_summary

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


measure                                      prevalence_of_zinc_deficiency_in_age_group_1_to_4  \
estimate                                                                                  mean   
location_id location_name                                                                        
168         Angola                                                                    0.141217   
161         Bangladesh                                                                0.096903   
201         Burkina Faso                                                              0.030959   
202         Cameroon                                                                  0.047513   
6           China                                                                     0.011407   
205         Côte d'Ivoire                                                             0.059209   
171         Democratic Republic of the Congo                                          0.325234   
141         Egypt                                                                     0.028587   
179         Ethiopia                                                                  0.028854   
207         Ghana                                                                     0.042126   
163         India                                                                     0.114851   
11          Indonesia                                                                 0.079103   
180         Kenya                                                                     0.051549   
181         Madagascar                                                                0.069270   
184         Mozambique                                                                0.139459   
15          Myanmar                                                                   0.060132   
164         Nepal                                                                     0.038835   
213         Niger                                                                     0.042498   
214         Nigeria                                                                   0.045405   
165         Pakistan                                                                  0.267418   
196         South Africa                                                              0.034747   
522         Sudan                                                                     0.337664   
190         Uganda                                                                    0.090285   
189         United Republic of Tanzania                                               0.068614   
20          Viet Nam                                                                  0.048809   
1           Global                                                                    0.087841   

measure                                                           \
estimate                                         lower     upper   
location_id location_name                                          
168         Angola                            0.025294  0.287093   
161         Bangladesh                        0.010825  0.231075   
201         Burkina Faso                      0.000000  0.098123   
202         Cameroon                          0.000000  0.137640   
6           China                             0.000000  0.033833   
205         Côte d'Ivoire                     0.002605  0.160208   
171         Democratic Republic of the Congo  0.180352  0.438343   
141         Egypt                             0.000000  0.092144   
179         Ethiopia                          0.000000  0.090044   
207         Ghana                             0.000000  0.125638   
163         India                             0.020721  0.240093   
11          Indonesia                         0.008081  0.188562   
180         Kenya                             0.001115  0.149923   
181         Madagascar                        0.004496  0.183884   
1

In [32]:
zinc_summary_filename='data/zinc_data_summary.csv'
zinc_summary.to_csv(zinc_summary_filename)
pd.read_csv(zinc_summary_filename, index_col=[0,1], header=[0,1]).head()

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


measure                   prevalence_of_zinc_deficiency_in_age_group_1_to_4  \
estimate                                                               mean   
location_id location_name                                                     
168         Angola                                                 0.141217   
161         Bangladesh                                             0.096903   
201         Burkina Faso                                           0.030959   
202         Cameroon                                               0.047513   
6           China                                                  0.011407   

measure                                        \
estimate                      lower     upper   
location_id location_name                       
168         Angola         0.025294  0.287093   
161         Bangladesh     0.010825  0.231075   
201         Burkina Faso   0.000000  0.098123   
202         Cameroon       0.000000  0.137640   
6           China          0.000000  0.033833   

measure                   number_in_age_group_1_to_4_with_zinc_deficiency  \
estimate                                                             mean   
location_id location_name                                                   
168         Angola                                           5.770476e+05   
161         Bangladesh                                       1.076757e+06   
201         Burkina Faso                                     9.782485e+04   
202         Cameroon                                         1.588382e+05   
6           China                                            7.577129e+05   

measure                                                 \
estimate                           lower         upper   
location_id location_name                                
168         Angola         103356.373557  1.173133e+06   
161         Bangladesh     120280.818575  2.567633e+06   
201         Burkina Faso        0.000000  3.100520e+05   
202         Cameroon            0.000000  4.601380e+05   
6           China               0.000000  2.247276e+06   

measure                   dalys_attributable_to_zinc_deficiency_in_age_group_1_to_4  \
estimate                                                                       mean   
location_id location_name                                                             
168         Angola                                               5150.114104          
161         Bangladesh                                            957.394251          
201         Burkina Faso                                         1693.518523          
202         Cameroon                                             3795.571168          
6           China                                                 221.834640          

measure                                              \
estimate                        lower         upper   
location_id location_name                             
168         Angola         767.607541  13767.079061   
161         Bangladesh      90.193038   2849.796608   
201         Burkina Faso     0.000000   6199.902944   
202         Cameroon         0.000000  13324.296023   
6           China            0.000000    743.348676   

measure                   dalys_attributable_to_zinc_deficiency_per_100000_py_in_age_group_1_to_4  \
estimate                                                                                     mean   
location_id location_name                                                                           
168         Angola                                                126.035134                        
161         Bangladesh                                              8.616112                        
201         Burkina Faso                                           53.595416                        
202         Cameroon                                              113.536157                        
6           China                              

In [89]:
zinc_summary.filter(like='prevalence')

measure                                      prevalence_of_zinc_deficiency_in_age_group_1_to_4  \
estimate                                                                                  mean   
location_id location_name                                                                        
168         Angola                                                                    0.141217   
161         Bangladesh                                                                0.096903   
201         Burkina Faso                                                              0.030959   
202         Cameroon                                                                  0.047513   
6           China                                                                     0.011407   
205         Côte d'Ivoire                                                             0.059209   
171         Democratic Republic of the Congo                                          0.325234   
141         Egypt                                                                     0.028587   
179         Ethiopia                                                                  0.028854   
207         Ghana                                                                     0.042126   
163         India                                                                     0.114851   
11          Indonesia                                                                 0.079103   
180         Kenya                                                                     0.051549   
181         Madagascar                                                                0.069270   
184         Mozambique                                                                0.139459   
15          Myanmar                                                                   0.060132   
164         Nepal                                                                     0.038835   
213         Niger                                                                     0.042498   
214         Nigeria                                                                   0.045405   
165         Pakistan                                                                  0.267418   
196         South Africa                                                              0.034747   
522         Sudan                                                                     0.337664   
190         Uganda                                                                    0.090285   
189         United Republic of Tanzania                                               0.068614   
20          Viet Nam                                                                  0.048809   
1           Global                                                                    0.087841   

measure                                                           
estimate                                         lower     upper  
location_id location_name                                         
168         Angola                            0.025294  0.287093  
161         Bangladesh                        0.010825  0.231075  
201         Burkina Faso                      0.000000  0.098123  
202         Cameroon                          0.000000  0.137640  
6           China                             0.000000  0.033833  
205         Côte d'Ivoire                     0.002605  0.160208  
171         Democratic Republic of the Congo  0.180352  0.438343  
141         Egypt                             0.000000  0.092144  
179         Ethiopia                          0.000000  0.090044  
207         Ghana                             0.000000  0.125638  
163         India                             0.020721  0.240093  
11          Indonesia                         0.008081  0.188562  
180         Kenya                             0.001115  0.149923  
181         Madagascar                        0.004496  0.183884  
184         Mozamb

# Check what's in my hdf file

In [33]:
with pd.HDFStore('data/lsff_gbd_data_summary.hdf', mode='r') as store:
    print(store.keys())

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['/zinc_deficiency/daly_burden', '/zinc_deficiency/population_1_to_4', '/zinc_deficiency/prevalence_among_1_to_4', '/vitamin_a_deficiency/dalys_attributable', '/vitamin_a_deficiency/population_under_5', '/vitamin_a_deficiency/prevalence_under_5']


In [34]:
vad_prevalence = pd.read_hdf('data/lsff_gbd_data_summary.hdf',
                              'vitamin_a_deficiency/prevalence_under_5')
vad_population = pd.read_hdf('data/lsff_gbd_data_summary.hdf',
                              'vitamin_a_deficiency/population_under_5')
vad_dalys = pd.read_hdf('data/lsff_gbd_data_summary.hdf',
                         'vitamin_a_deficiency/dalys_attributable')

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/tables/atom.py:1224: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return pickle.loads(array.tostring())


In [35]:
vad_dalys.head()

,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,metric_id,rei_id,sex_id,year_id
0,2,294,610.265537,7.129188e+02,584.278363,576.937717,409.973505,607.233541,728.870375,493.579920,...,248.213171,486.247220,438.085573,436.432390,1,2,1,96,2,2019
1,3,294,9363.198293,1.092949e+04,8368.736002,9982.006511,7609.211513,9183.636889,10542.066104,7324.188474,...,4369.326708,7280.572863,7897.450957,6851.663011,1,2,1,96,2,2019
2,4,294,449406.675608,5.545513e+05,338516.272279,590073.349785,126476.979025,158563.491568,136273.580110,484981.100751,...,244008.653919,380229.833318,274865.340592,152344.707050,1,2,1,96,2,2019
3,5,294,854904.099987,1.071534e+06,583597.902356,902017.353676,259573.288255,311673.209653,296055.171332,839221.584272,...,389873.831466,496395.231429,569553.194295,298005.258843,1,2,1,96,2,2019
4,6,294,179144.094706,1.526072e+05,115113.429804,168106.134250,116613.233007,116600.455885,153721.578560,106277.390857,...,61805.345403,122170.134996,118225.047839,110907.411543,1,2,1,96,2,2019


In [36]:
vad_prevalence.head()

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,parameter,measure_id,metric_id,draw_0,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
0,96,2510.0,1,2019,1,3,cat1,18,3,0.160723,...,0.138674,0.156027,0.162336,0.145321,0.148542,0.144228,0.150515,0.163942,0.140099,0.163181
1,96,2510.0,6,2019,1,3,cat1,18,3,0.025171,...,0.024486,0.041571,0.054358,0.036757,0.049439,0.048223,0.033412,0.019751,0.024105,0.055701
2,96,2510.0,11,2019,1,3,cat1,18,3,0.135798,...,0.200273,0.168939,0.174769,0.142576,0.160275,0.142429,0.159862,0.158847,0.143874,0.144510
3,96,2510.0,15,2019,1,3,cat1,18,3,0.120545,...,0.098860,0.159518,0.162904,0.131492,0.123332,0.123488,0.101929,0.180399,0.183362,0.167684
4,96,2510.0,20,2019,1,3,cat1,18,3,0.097730,...,0.076831,0.058292,0.058919,0.107750,0.111074,0.102168,0.067863,0.077999,0.073056,0.102493


In [38]:
vad_population.head()

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,age_group_id,location_id,year_id,sex_id,population,upper,lower,run_id
0,1,1,2019,3,6.628427e+08,6.819745e+08,6.438792e+08,192
1,1,6,2019,3,8.149086e+07,9.152573e+07,7.098894e+07,192
2,1,11,2019,3,1.970635e+07,2.217715e+07,1.722869e+07,192
3,1,15,2019,3,5.018670e+06,5.528959e+06,4.489139e+06,192
4,1,20,2019,3,6.937449e+06,7.846919e+06,5.979573e+06,192


# Test shared functions for neural tube defects

In [41]:
test_location_ids = [15, 164] #Myanmar, Nepal

In [43]:
%%time
ntd_dalys = get_draws(
        gbd_id_type='cause_id',
        gbd_id=list_ids('cause', 'Neural tube defects'),
        source='dalynator',
        measure_id=find_ids('measure', 'DALYs'),
#         metric_id=list_ids('metric', 'Number', 'Percent'),
        location_id=test_location_ids,
        year_id=2019,
        sex_id=list_ids('sex', 'Male', 'Female', 'Both'),
        age_group_id=list_ids('age_group', 'Under 5'),
        gbd_round_id=list_ids('gbd_round', '2019'),
        status='best',
        decomp_step='step5',
    )
ntd_dalys.head()

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 18.2 s, sys: 1.46 s, total: 19.7 s
Wall time: 26 s


,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,metric_id,sex_id,year_id
0,1,642,0.012143,0.017883,0.008016,0.010273,0.009223,0.020538,0.006648,0.007467,...,0.010959,0.015911,0.017902,0.014416,0.010359,15,2,3,1,2019
1,1,642,0.007402,0.027347,0.011419,0.008061,0.011936,0.018012,0.008999,0.014890,...,0.007889,0.009454,0.018339,0.013660,0.019215,15,2,3,2,2019
2,1,642,0.009823,0.022515,0.009682,0.009190,0.010551,0.019301,0.007799,0.011100,...,0.009456,0.012750,0.018116,0.014046,0.014694,15,2,3,3,2019
3,1,642,0.010602,0.019600,0.009560,0.010849,0.009380,0.026040,0.007733,0.006914,...,0.011632,0.018269,0.018146,0.019371,0.015868,15,2,2,1,2019
4,1,642,0.008303,0.037343,0.016864,0.010629,0.015285,0.027979,0.013100,0.017499,...,0.010516,0.013583,0.023613,0.022401,0.035572,15,2,2,2,2019


In [46]:
ids_to_names('metric', *ntd_dalys.metric_id.unique())

metric_id
1     Number
2    Percent
3       Rate
Name: metric_name, dtype: object

In [48]:
ntd_population = get_population(
        age_group_id=list_ids('age_group', 'Birth', 'Under 5'),
        sex_id=list_ids('sex', 'Male', 'Female', 'Both'),
        location_id=test_location_ids,
        year_id=2019,
        gbd_round_id=list_ids('gbd_round', '2019'),
        decomp_step='step4',
        with_ui=True,
    )
ntd_population

,age_group_id,location_id,year_id,sex_id,population,upper,lower,run_id
0,1,15,2019,1,2.562164e+06,2.822681e+06,2.291825e+06,192
1,164,15,2019,1,5.414443e+05,6.127138e+05,4.775972e+05,192
2,1,15,2019,2,2.456506e+06,2.706279e+06,2.197314e+06,192
3,164,15,2019,2,5.134675e+05,5.810545e+05,4.529194e+05,192
4,1,15,2019,3,5.018670e+06,5.528959e+06,4.489139e+06,192
5,164,15,2019,3,1.054912e+06,1.193768e+06,9.305166e+05,192
6,1,164,2019,1,1.539103e+06,1.732521e+06,1.346567e+06,192
7,164,164,2019,1,3.181367e+05,3.624567e+05,2.813146e+05,192
8,1,164,2019,2,1.456865e+06,1.639949e+06,1.274617e+06,192
9,164,164,2019,2,2.999425e+05,3.417279e+05,2.652263e+05,192


## Pull NTD data, save to my `.hdf` file, and preprocess for the data summary

In [50]:
%%time
ntd_data = pgd.pull_neural_tube_defects_summary_input_data_for_locations(
    locations.location_id, hdf_filepath)
len(ntd_data)

CPU times: user 3min 35s, sys: 17.3 s, total: 3min 52s
Wall time: 4min 52s


3

In [57]:
ntd_preprocessed = pgd.preprocess_neural_tube_defects_data(ntd_data)
len(ntd_preprocessed)

5

In [59]:
ntd_preprocessed.daly_count

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,metric_id,sex_id,year_id
8,1,642,5.970445e+06,6.387010e+06,6.108604e+06,7.404369e+06,6.013689e+06,5.762819e+06,5.136962e+06,6.154354e+06,...,9.133882e+06,8.097010e+06,9.970295e+06,8.110397e+06,1.009944e+07,1,2,1,3,2019
17,1,642,7.116025e+04,7.804144e+04,7.730261e+04,1.473221e+05,1.371189e+05,1.091769e+05,1.011687e+05,1.043757e+05,...,1.368378e+05,1.540637e+05,1.530902e+05,1.195830e+05,1.581032e+05,11,2,1,3,2019
26,1,642,2.486852e+04,4.900662e+04,8.216102e+04,5.442871e+04,2.113676e+04,2.791546e+04,2.928933e+04,3.012447e+04,...,8.187806e+04,4.212335e+04,5.155310e+04,5.343899e+04,5.326659e+04,141,2,1,3,2019
35,1,642,4.929701e+04,1.129964e+05,4.859065e+04,4.612310e+04,5.295202e+04,9.686702e+04,3.914012e+04,5.570949e+04,...,4.745856e+04,6.398923e+04,9.091924e+04,7.049238e+04,7.374320e+04,15,2,1,3,2019
44,1,642,8.144175e+04,7.643334e+04,5.582812e+04,1.249708e+05,1.079833e+05,1.615961e+05,6.812759e+04,2.275002e+05,...,6.581571e+04,7.218776e+04,5.613237e+04,6.695543e+04,8.167159e+04,161,2,1,3,2019
53,1,642,5.056808e+05,8.971136e+05,6.253296e+05,9.860218e+05,8.147714e+05,7.193659e+05,7.154149e+05,7.775107e+05,...,5.293013e+05,3.904036e+05,6.205005e+05,4.404073e+05,6.306940e+05,163,2,1,3,2019
62,1,642,2.407000e+04,2.193964e+04,1.884085e+04,1.609759e+04,1.087132e+04,1.592076e+04,7.288049e+03,1.397640e+04,...,7.562728e+03,9.667764e+03,1.103587e+04,4.735283e+03,1.035855e+04,164,2,1,3,2019
71,1,642,2.768024e+05,1.957462e+05,1.983214e+05,3.170881e+05,2.994353e+05,2.767873e+05,2.340487e+05,2.774027e+05,...,1.532803e+05,2.073852e+05,1.988117e+05,2.083422e+05,2.908698e+05,165,2,1,3,2019
80,1,642,6.666548e+04,9.583775e+04,1.259556e+05,9.690449e+04,6.275474e+04,9.788391e+04,5.791835e+04,7.595988e+04,...,1.730039e+05,1.575076e+05,1.893149e+05,1.549935e+05,1.212606e+05,168,2,1,3,2019
89,1,642,3.763814e+05,2.944268e+05,2.453879e+05,2.988728e+05,1.364877e+05,1.690887e+05,1.422119e+05,3.350577e+05,...,5.191276e+05,2.638715e+05,3.943540e+05,2.570453e+05,4.055070e+05,171,2,1,3,2019


In [60]:
ntd_preprocessed.daly_rate

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,metric_id,sex_id,year_id
2,1,642,0.009007,0.009636,0.009216,0.011171,0.009073,0.008694,0.007750,0.009285,...,0.013780,0.012216,0.015042,0.012236,0.015237,1,2,3,3,2019
11,1,642,0.003611,0.003960,0.003923,0.007476,0.006958,0.005540,0.005134,0.005297,...,0.006944,0.007818,0.007769,0.006068,0.008023,11,2,3,3,2019
20,1,642,0.002297,0.004527,0.007590,0.005028,0.001953,0.002579,0.002706,0.002783,...,0.007564,0.003891,0.004762,0.004937,0.004921,141,2,3,3,2019
29,1,642,0.009823,0.022515,0.009682,0.009190,0.010551,0.019301,0.007799,0.011100,...,0.009456,0.012750,0.018116,0.014046,0.014694,15,2,3,3,2019
38,1,642,0.005925,0.005560,0.004061,0.009091,0.007856,0.011756,0.004956,0.016550,...,0.004788,0.005252,0.004084,0.004871,0.005942,161,2,3,3,2019
47,1,642,0.004319,0.007663,0.005341,0.008422,0.006960,0.006145,0.006111,0.006641,...,0.004521,0.003335,0.005300,0.003762,0.005387,163,2,3,3,2019
56,1,642,0.008034,0.007323,0.006289,0.005373,0.003629,0.005314,0.002433,0.004665,...,0.002524,0.003227,0.003684,0.001581,0.003457,164,2,3,3,2019
65,1,642,0.009068,0.006412,0.006497,0.010387,0.009809,0.009067,0.007667,0.009087,...,0.005021,0.006794,0.006513,0.006825,0.009528,165,2,3,3,2019
74,1,642,0.012952,0.018620,0.024471,0.018827,0.012192,0.019017,0.011253,0.014758,...,0.033612,0.030601,0.036781,0.030112,0.023559,168,2,3,3,2019
83,1,642,0.027615,0.021602,0.018004,0.021928,0.010014,0.012406,0.010434,0.024583,...,0.038089,0.019360,0.028934,0.018860,0.029752,171,2,3,3,2019


In [61]:
ntd_preprocessed.population

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,age_group_id,location_id,year_id,sex_id,population,upper,lower,run_id
5,164,1,2019,3,1.353500e+08,1.440812e+08,1.271671e+08,192
11,164,6,2019,3,1.492184e+07,1.535535e+07,1.446737e+07,192
17,164,11,2019,3,3.837830e+06,4.287325e+06,3.457880e+06,192
23,164,15,2019,3,1.054912e+06,1.193768e+06,9.305166e+05,192
29,164,20,2019,3,1.340431e+06,1.498616e+06,1.213495e+06,192
35,164,141,2019,3,2.107666e+06,2.345984e+06,1.879636e+06,192
41,164,161,2019,3,2.659503e+06,2.945596e+06,2.378522e+06,192
47,164,163,2019,3,2.346104e+07,2.577264e+07,2.114538e+07,192
53,164,164,2019,3,6.180792e+05,7.041846e+05,5.465409e+05,192
59,164,165,2019,3,6.686023e+06,7.548104e+06,5.941982e+06,192


In [62]:
ntd_preprocessed.prevalence

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,sex_id,year_id,metric_id
488,164,642.0,0.000875,0.000955,0.001016,0.001200,0.001214,0.001037,0.001077,0.001296,...,0.000907,0.001066,0.000912,0.001146,0.001155,1,6.0,3,2019,3.0
512,164,642.0,0.000244,0.000258,0.000267,0.000304,0.000327,0.000256,0.000270,0.000336,...,0.000238,0.000276,0.000260,0.000289,0.000292,6,6.0,3,2019,3.0
536,164,642.0,0.000344,0.000362,0.000398,0.000432,0.000460,0.000388,0.000397,0.000480,...,0.000347,0.000407,0.000372,0.000451,0.000444,11,6.0,3,2019,3.0
560,164,642.0,0.000921,0.000956,0.001007,0.001107,0.001314,0.000998,0.001095,0.001316,...,0.000917,0.001144,0.000948,0.001179,0.001093,15,6.0,3,2019,3.0
584,164,642.0,0.000343,0.000387,0.000405,0.000435,0.000413,0.000380,0.000386,0.000460,...,0.000350,0.000373,0.000364,0.000401,0.000402,20,6.0,3,2019,3.0
608,164,642.0,0.000743,0.000828,0.000848,0.000971,0.000946,0.000840,0.000944,0.001000,...,0.000762,0.000947,0.000810,0.000970,0.000953,141,6.0,3,2019,3.0
632,164,642.0,0.000697,0.000810,0.000797,0.000904,0.000963,0.000821,0.000851,0.001084,...,0.000737,0.000837,0.000766,0.000921,0.001002,161,6.0,3,2019,3.0
656,164,642.0,0.000801,0.000916,0.000965,0.001110,0.001141,0.000962,0.001008,0.001250,...,0.000858,0.000978,0.000883,0.001099,0.001101,163,6.0,3,2019,3.0
680,164,642.0,0.000592,0.000692,0.000681,0.000818,0.000828,0.000720,0.000703,0.000873,...,0.000617,0.000742,0.000679,0.000765,0.000796,164,6.0,3,2019,3.0
704,164,642.0,0.000779,0.000828,0.000916,0.000989,0.001083,0.000932,0.000977,0.001165,...,0.000840,0.000975,0.000816,0.001149,0.001112,165,6.0,3,2019,3.0


In [72]:
ntd_data[0].tail()

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,sex_id,year_id,metric_id
1085,30,642.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,522,6.0,3,2019,3.0
1086,31,642.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,522,6.0,3,2019,3.0
1087,32,642.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,522,6.0,3,2019,3.0
1088,164,642.0,0.001829,0.002234,0.002283,0.002702,0.002826,0.002404,0.002651,0.003025,...,0.001908,0.002416,0.001997,0.002748,0.002584,522,6.0,3,2019,3.0
1089,235,642.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,522,6.0,3,2019,3.0


In [68]:
ntd_data[0].shape

(1090, 1007)

## Above, it looks like there are many extraneous rows with 0's in the raw NTD data

## Try calling `get_draws` with different parameters to get just what we need

It looks like perhaps including the 'Global' location causes `get_draws` to pull incidence data for all age groups, even though it's all 0 except in age group 164.

In [71]:
%%time
ntd_bp = get_draws(
        gbd_id_type='cause_id',
        gbd_id=list_ids('cause', 'Neural tube defects'),
        source='como',
        location_id=test_location_ids,
        year_id=2019,
#         age_group_id=list_ids('age_group', 'Birth'),
        sex_id=list_ids('sex', 'Both'),
        measure_id=list_ids('measure', 'Incidence'),
        gbd_round_id=list_ids('gbd_round', '2019'),
        status='best',
        decomp_step='step5',
    )
ntd_bp.head()

CPU times: user 5.45 s, sys: 339 ms, total: 5.79 s
Wall time: 9.09 s


,metric_id,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,...,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,sex_id,year_id
0,3,164.0,642.0,0.000921,0.000956,0.001007,0.001107,0.001314,0.000998,0.001095,...,0.000995,0.000917,0.001144,0.000948,0.001179,0.001093,15.0,6.0,3.0,2019.0
1,3,164.0,642.0,0.000592,0.000692,0.000681,0.000818,0.000828,0.000720,0.000703,...,0.000602,0.000617,0.000742,0.000679,0.000765,0.000796,164.0,6.0,3.0,2019.0


In [75]:
ntd_bp.dtypes

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


metric_id         int64
age_group_id    float64
cause_id        float64
draw_0          float64
draw_1          float64
                 ...   
draw_999        float64
location_id     float64
measure_id      float64
sex_id          float64
year_id         float64
Length: 1007, dtype: object

In [73]:
ntd_bp.shape

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(2, 1007)

In [74]:
%%time
ntd_bp2 = get_draws(
        gbd_id_type='cause_id',
        gbd_id=list_ids('cause', 'Neural tube defects'),
        source='como',
        location_id=test_location_ids,
        year_id=2019,
#         age_group_id=list_ids('age_group', 'Birth'),
        sex_id=list_ids('sex', 'Male', 'Female', 'Both'),
        measure_id=list_ids('measure', 'Incidence'),
        gbd_round_id=list_ids('gbd_round', '2019'),
        status='best',
        decomp_step='step5',
    )
ntd_bp2.head()

CPU times: user 3.74 s, sys: 177 ms, total: 3.91 s
Wall time: 5.91 s


,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,sex_id,year_id,metric_id
0,164,642,0.000983,0.000996,0.001069,0.001095,0.001351,0.001023,0.001030,0.001337,...,0.000967,0.001186,0.000934,0.001227,0.001122,15,6,1,2019,3
1,164,642,0.000856,0.000914,0.000941,0.001118,0.001274,0.000972,0.001163,0.001294,...,0.000864,0.001099,0.000964,0.001129,0.001064,15,6,2,2019,3
2,164,642,0.000516,0.000564,0.000568,0.000677,0.000674,0.000598,0.000560,0.000723,...,0.000522,0.000619,0.000573,0.000569,0.000643,164,6,1,2019,3
3,164,642,0.000673,0.000828,0.000802,0.000968,0.000992,0.000849,0.000855,0.001031,...,0.000718,0.000873,0.000792,0.000973,0.000958,164,6,2,2019,3
4,164,642,0.000921,0.000956,0.001007,0.001107,0.001314,0.000998,0.001095,0.001316,...,0.000917,0.001144,0.000948,0.001179,0.001093,15,6,3,2019,3


In [79]:
ntd_bp2.shape

(6, 1007)

In [76]:
ntd_bp2.dtypes

age_group_id      int64
cause_id          int64
draw_0          float64
draw_1          float64
draw_10         float64
                 ...   
location_id       int64
measure_id        int64
sex_id            int64
year_id           int64
metric_id         int64
Length: 1007, dtype: object

In [77]:
%%time
ntd_bp3 = get_draws(
        gbd_id_type='cause_id',
        gbd_id=list_ids('cause', 'Neural tube defects'),
        source='como',
        location_id=[1, *test_location_ids],
        year_id=2019,
#         age_group_id=list_ids('age_group', 'Birth'),
        sex_id=list_ids('sex', 'Male', 'Female', 'Both'),
        measure_id=list_ids('measure', 'Incidence'),
        gbd_round_id=list_ids('gbd_round', '2019'),
        status='best',
        decomp_step='step5',
    )
ntd_bp3.head()

CPU times: user 4.05 s, sys: 196 ms, total: 4.25 s
Wall time: 7.04 s


,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,sex_id,year_id,metric_id
0,2,642.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,6.0,1,2019,3.0
1,3,642.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,6.0,1,2019,3.0
2,4,642.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,6.0,1,2019,3.0
3,5,642.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,6.0,1,2019,3.0
4,6,642.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,6.0,1,2019,3.0


In [78]:
ntd_bp3.shape

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(124, 1007)

In [84]:
ids_to_names('metric', *ntd_bp3.metric_id.unique())

metric_id
3    Rate
Name: metric_name, dtype: object

In [80]:
test_location_ids

[15, 164]

In [81]:
ntd_bp3.query("location_id==15")

,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,sex_id,year_id,metric_id
48,164,642.0,0.000983,0.000996,0.001069,0.001095,0.001351,0.001023,0.001030,0.001337,...,0.000967,0.001186,0.000934,0.001227,0.001122,15,6.0,1,2019,3.0
49,164,642.0,0.000856,0.000914,0.000941,0.001118,0.001274,0.000972,0.001163,0.001294,...,0.000864,0.001099,0.000964,0.001129,0.001064,15,6.0,2,2019,3.0
76,2,642.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,15,6.0,3,2019,3.0
77,3,642.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,15,6.0,3,2019,3.0
78,4,642.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,15,6.0,3,2019,3.0
79,5,642.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,15,6.0,3,2019,3.0
80,6,642.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,15,6.0,3,2019,3.0
81,7,642.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,15,6.0,3,2019,3.0
82,8,642.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,15,6.0,3,2019,3.0
83,9,642.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,15,6.0,3,2019,3.0


In [88]:
ntd_bp3.query("location_id==1").shape

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(72, 1007)

In [87]:
ntd_bp3.query("location_id==1").loc[ntd_bp3.query("location_id==1").draw_0>0]

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,sex_id,year_id,metric_id
22,164,642.0,0.000783,0.000864,0.000918,0.001074,0.001097,0.000928,0.000967,0.001169,...,0.000821,0.000964,0.000827,0.001034,0.001040,1,6.0,1,2019,3.0
46,164,642.0,0.000974,0.001054,0.001121,0.001335,0.001341,0.001154,0.001195,0.001433,...,0.000998,0.001176,0.001003,0.001265,0.001279,1,6.0,2,2019,3.0
74,164,642.0,0.000875,0.000955,0.001016,0.001200,0.001214,0.001037,0.001077,0.001296,...,0.000907,0.001066,0.000912,0.001146,0.001155,1,6.0,3,2019,3.0


In [83]:
ntd_bp3.query("location_id==1").age_group_id.unique()

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  30,  31,  32, 164, 235])

# Make NTD summary

In [105]:
!ls data

all_countries_with_ids.csv
bgmf_countries.csv
bgmf_countries_with_ids.csv
bmgf_top_25_countries_20201203.csv
countries_ranked_by_number_stunted_2019.csv
countries_with_over_20_percent_stunting_ranked_by_number_stunted_2019.csv
lsff_gbd_data_summary.hdf
neural_tube_defects_output_gbd2017.csv
percent_global_burden_mn_deficiency_for_top25.csv
stunting_prevalence_2019.hdf
top_25_countries_with_stunting_above_18_percent_in_2019.csv
top_25_countries_with_stunting_above_20_percent_in_2019.csv
vitamin_a_data_summary.csv
zinc_data_summary.csv


/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [106]:
ntd_preprocessed = pgd.preprocess_neural_tube_defects_data(ntd_data)

In [108]:
ntd_summary = pgd.get_binary_outcome_data_summary(
    ntd_preprocessed, 'Neural tube defects', save_filepath='data/neural_tube_defects_data_summary.csv')
ntd_summary

measure                                      Prevalence of Neural tube defects in age group Birth  \
estimate                                                                                     mean   
location_id location_name                                                                           
168         Angola                                                                     0.001819     
161         Bangladesh                                                                 0.000816     
201         Burkina Faso                                                               0.002470     
202         Cameroon                                                                   0.001834     
6           China                                                                      0.000264     
205         Côte d'Ivoire                                                              0.002211     
171         Democratic Republic of the Congo                                           0.002620     
141         Egypt                                                                      0.000858     
179         Ethiopia                                                                   0.001121     
207         Ghana                                                                      0.001403     
163         India                                                                      0.000943     
11          Indonesia                                                                  0.000382     
180         Kenya                                                                      0.000952     
181         Madagascar                                                                 0.001457     
184         Mozambique                                                                 0.001723     
15          Myanmar                                                                    0.001012     
164         Nepal                                                                      0.000691     
213         Niger                                                                      0.001794     
214         Nigeria                                                                    0.002393     
165         Pakistan                                                                   0.000899     
196         South Africa                                                               0.000544     
522         Sudan                                                                      0.002277     
190         Uganda                                                                     0.002048     
189         United Republic of Tanzania                                                0.001799     
20          Viet Nam                                                                   0.000372     
1           Global                                                                     0.001005     

measure                                                           \
estimate                                         lower     upper   
location_id location_name                                          
168         Angola                            0.001417  0.002291   
161         Bangladesh                        0.000643  0.001019   
201         Burkina Faso                      0.001913  0.003151   
202         Cameroon                          0.001431  0.002403   
6           China                             0.000213  0.000325   
205         Côte d'Ivoire                     0.001763  0.002842   
171         Democratic Republic of the Congo  0.002022  0.003334   
141         Egypt                             0.000688  0.001068   
179         Ethiopia                          0.000868  0.001434   
207         Ghana                             0.001123  0.001749   
163         India                             0.000745  0.001197   
11          Indonesia                         0.000303  0.000480   
180         Kenya                             0.00

# Recalculate VAD data

In [109]:
vad_data = (vad_prevalence, vad_population, vad_dalys)
vad_preprocessed = pgd.preprocess_binary_risk_summary_input_data(vad_data, 'Vitamin A deficiency')

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [110]:
%%time
vad_summary = pgd.get_binary_outcome_data_summary(
    vad_preprocessed,  'Vitamin A deficiency', save_filepath='data/vitamin_a_data_summary.csv')
vad_summary

CPU times: user 378 ms, sys: 10 ms, total: 388 ms
Wall time: 397 ms


measure                                      Prevalence of Vitamin A deficiency in age group Under 5  \
estimate                                                                                        mean   
location_id location_name                                                                              
168         Angola                                                                     0.263851        
161         Bangladesh                                                                 0.084110        
201         Burkina Faso                                                               0.354247        
202         Cameroon                                                                   0.279080        
6           China                                                                      0.038543        
205         Côte d'Ivoire                                                              0.261877        
171         Democratic Republic of the Congo                                           0.514314        
141         Egypt                                                                      0.029521        
179         Ethiopia                                                                   0.356954        
207         Ghana                                                                      0.266086        
163         India                                                                      0.168170        
11          Indonesia                                                                  0.166394        
180         Kenya                                                                      0.468136        
181         Madagascar                                                                 0.365863        
184         Mozambique                                                                 0.413208        
15          Myanmar                                                                    0.146447        
164         Nepal                                                                      0.096260        
213         Niger                                                                      0.646669        
214         Nigeria                                                                    0.080588        
165         Pakistan                                                                   0.068985        
196         South Africa                                                               0.108419        
522         Sudan                                                                      0.195104        
190         Uganda                                                                     0.215768        
189         United Republic of Tanzania                                                0.215579        
20          Viet Nam                                                                   0.085598        
1           Global                                                                     0.151638        

measure                                                           \
estimate                                         lower     upper   
location_id location_name                                          
168         Angola                            0.183993  0.368989   
161         Bangladesh                        0.052888  0.128375   
201         Burkina Faso                      0.260300  0.456761   
202         Cameroon                          0.189659  0.384803   
6           China                             0.023616  0.061301   
205         Côte d'Ivoire                     0.182838  0.367594   
171         Democratic Republic of the Congo  0.405561  0.624600   
141         Egypt                             0.018288  0.045664   
179         Ethiopia                          0.258226  0.474010   
207         Ghana                             0.178249  0.364197   
163         India                             0.111292  0.252074   
11          Indonesia          

# Recalculate zinc deficiency data

In [111]:
zinc_data = (zinc_prevalence, zinc_population, zinc_burden)
zinc_preprocessed = pgd.preprocess_binary_risk_summary_input_data(zinc_data, 'Zinc deficiency')

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [112]:
%%time
zinc_summary = pgd.get_binary_outcome_data_summary(
    zinc_preprocessed, 'Zinc deficiency', save_filepath='data/zinc_data_summary.csv')
zinc_summary

CPU times: user 419 ms, sys: 877 µs, total: 420 ms
Wall time: 434 ms


measure                                      Prevalence of Zinc deficiency in age group 1 to 4  \
estimate                                                                                  mean   
location_id location_name                                                                        
168         Angola                                                                    0.141217   
161         Bangladesh                                                                0.096903   
201         Burkina Faso                                                              0.030959   
202         Cameroon                                                                  0.047513   
6           China                                                                     0.011407   
205         Côte d'Ivoire                                                             0.059209   
171         Democratic Republic of the Congo                                          0.325234   
141         Egypt                                                                     0.028587   
179         Ethiopia                                                                  0.028854   
207         Ghana                                                                     0.042126   
163         India                                                                     0.114851   
11          Indonesia                                                                 0.079103   
180         Kenya                                                                     0.051549   
181         Madagascar                                                                0.069270   
184         Mozambique                                                                0.139459   
15          Myanmar                                                                   0.060132   
164         Nepal                                                                     0.038835   
213         Niger                                                                     0.042498   
214         Nigeria                                                                   0.045405   
165         Pakistan                                                                  0.267418   
196         South Africa                                                              0.034747   
522         Sudan                                                                     0.337664   
190         Uganda                                                                    0.090285   
189         United Republic of Tanzania                                               0.068614   
20          Viet Nam                                                                  0.048809   
1           Global                                                                    0.087841   

measure                                                           \
estimate                                         lower     upper   
location_id location_name                                          
168         Angola                            0.025294  0.287093   
161         Bangladesh                        0.010825  0.231075   
201         Burkina Faso                      0.000000  0.098123   
202         Cameroon                          0.000000  0.137640   
6           China                             0.000000  0.033833   
205         Côte d'Ivoire                     0.002605  0.160208   
171         Democratic Republic of the Congo  0.180352  0.438343   
141         Egypt                             0.000000  0.092144   
179         Ethiopia                          0.000000  0.090044   
207         Ghana                             0.000000  0.125638   
163         India                             0.020721  0.240093   
11          Indonesia                         0.008081  0.188562   
180         Kenya                             0.001115  0.149923   
181         Madagascar                        0.004496  0.183884   
1

# Try formatting "mean (lower, upper)"

In [117]:
zprev = zinc_summary.filter(regex=r'Prevalence')
zdalys = zinc_summary.filter(regex=r'DALYs')
zdalys

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


measure                                      DALYs attributable to Zinc deficiency in age group 1 to 4  \
estimate                                                                                          mean   
location_id location_name                                                                                
168         Angola                                                                  5150.114104          
161         Bangladesh                                                               957.394251          
201         Burkina Faso                                                            1693.518523          
202         Cameroon                                                                3795.571168          
6           China                                                                    221.834640          
205         Côte d'Ivoire                                                           1627.155879          
171         Democratic Republic of the Congo                                       28539.815244          
141         Egypt                                                                    719.287717          
179         Ethiopia                                                                4194.901787          
207         Ghana                                                                    674.080086          
163         India                                                                  31741.385904          
11          Indonesia                                                               2825.142448          
180         Kenya                                                                   1708.764677          
181         Madagascar                                                              3974.917946          
184         Mozambique                                                              2552.568020          
15          Myanmar                                                                  345.172096          
164         Nepal                                                                    126.290884          
213         Niger                                                                   6970.558379          
214         Nigeria                                                                38748.277862          
165         Pakistan                                                               41008.892461          
196         South Africa                                                             357.746361          
522         Sudan                                                                   7362.702051          
190         Uganda                                                                  2313.981531          
189         United Republic of Tanzania                                             1453.807908          
20          Viet Nam                                                                 112.913147          
1           Global                                                                258812.803459          

measure                                                                    \
estimate                                             lower          upper   
location_id location_name                                                   
168         Angola                              767.607541   13767.079061   
161         Bangladesh                           90.193038    2849.796608   
201         Burkina Faso                          0.000000    6199.902944   
202         Cameroon                              0.000000   13324.296023   
6           China                                 0.000000     743.348676   
205         Côte d'Ivoire                        38.962458    6068.165571   
171         Democratic Republic of the Congo   4984.407705   79052.150750   
141         Egypt                                 0.000000    2487.810635   
179         Ethiopia                              0.000000   15205.492988   
2

In [115]:
idx=pd.IndexSlice

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [119]:
zdalys.columns.get_level_values('measure')

Index(['DALYs attributable to Zinc deficiency in age group 1 to 4',
       'DALYs attributable to Zinc deficiency in age group 1 to 4',
       'DALYs attributable to Zinc deficiency in age group 1 to 4',
       'DALYs attributable to Zinc deficiency per 100,000 person-years in age group 1 to 4',
       'DALYs attributable to Zinc deficiency per 100,000 person-years in age group 1 to 4',
       'DALYs attributable to Zinc deficiency per 100,000 person-years in age group 1 to 4'],
      dtype='object', name='measure')

In [126]:
def print_mean_lower_upper(row):
    return f"{row['mean']} ({row['lower']}, {row['upper']})"

cols = []
for measure in zdalys.columns.get_level_values('measure').unique():
    df = zdalys[measure]
    cols.append(df.apply(print_mean_lower_upper, axis=1).rename(measure))
pd.concat(cols, axis=1)

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,,DALYs attributable to Zinc deficiency in age group 1 to 4,"DALYs attributable to Zinc deficiency per 100,000 person-years in age group 1 to 4"
location_id,location_name,,
168,Angola,"5150.114103536395 (767.607541280017, 13767.079...","126.0351337853214 (18.78512149729208, 336.9120..."
161,Bangladesh,"957.3942506410937 (90.19303790344559, 2849.796...","8.61611165191563 (0.8116962100839561, 25.64686..."
201,Burkina Faso,"1693.5185227413351 (0.0, 6199.902943540811)","53.59541573703625 (0.0, 196.2106533387456)"
202,Cameroon,"3795.571168301141 (0.0, 13324.296023147486)","113.53615731728148 (0.0, 398.5669882994741)"
6,China,"221.83464000681988 (0.0, 743.3486755289681)","0.33397330291078003 (0.0, 1.119115627627547)"
205,Côte d'Ivoire,"1627.1558787983124 (38.96245804334615, 6068.16...","51.3954969111335 (1.2306718232157603, 191.6696..."
171,Democratic Republic of the Congo,"28539.815243649093 (4984.407704894977, 79052.1...","263.7410896025733 (46.06172492321925, 730.5338..."
141,Egypt,"719.2877171528244 (0.0, 2487.8106354141146)","8.239989082689842 (0.0, 28.49976718183901)"
179,Ethiopia,"4194.901786515336 (0.0, 15205.492988162932)","31.828721316879506 (0.0, 115.37133011355508)"


# Generate formatted summaries

In [144]:
pgd.format_data_summary(zinc_summary, save_filename='data/zinc_data_formatted.csv')

,,Prevalence of Zinc deficiency in age group 1 to 4 (per 100),Number in age group 1 to 4 with Zinc deficiency (1000s),DALYs attributable to Zinc deficiency in age group 1 to 4,"DALYs attributable to Zinc deficiency per 100,000 person-years in age group 1 to 4"
location_id,location_name,,,,
168,Angola,"14.1\n(2.5, 28.7)","577\n(103, 1,173)","5,150\n(768, 13,767)","126\n(19, 337)"
161,Bangladesh,"9.7\n(1.1, 23.1)","1,077\n(120, 2,568)","957\n(90, 2,850)","9\n(1, 26)"
201,Burkina Faso,"3.1\n(0.0, 9.8)","98\n(0, 310)","1,694\n(0, 6,200)","54\n(0, 196)"
202,Cameroon,"4.8\n(0.0, 13.8)","159\n(0, 460)","3,796\n(0, 13,324)","114\n(0, 399)"
6,China,"1.1\n(0.0, 3.4)","758\n(0, 2,247)","222\n(0, 743)","0\n(0, 1)"
205,Côte d'Ivoire,"5.9\n(0.3, 16.0)","187\n(8, 507)","1,627\n(39, 6,068)","51\n(1, 192)"
171,Democratic Republic of the Congo,"32.5\n(18.0, 43.8)","3,519\n(1,952, 4,743)","28,540\n(4,984, 79,052)","264\n(46, 731)"
141,Egypt,"2.9\n(0.0, 9.2)","250\n(0, 804)","719\n(0, 2,488)","8\n(0, 28)"
179,Ethiopia,"2.9\n(0.0, 9.0)","380\n(0, 1,187)","4,195\n(0, 15,205)","32\n(0, 115)"


In [147]:
pgd.format_data_summary(vad_summary,
                        daly_count_units=1000,
                        daly_count_decimals=1,
                        save_filename='data/vitamin_a_data_formatted.csv')

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,,Prevalence of Vitamin A deficiency in age group Under 5 (per 100),Number in age group Under 5 with Vitamin A deficiency (1000s),DALYs attributable to Vitamin A deficiency in age group Under 5 (1000s),"DALYs attributable to Vitamin A deficiency per 100,000 person-years in age group Under 5"
location_id,location_name,,,,
168,Angola,"26.4\n(18.4, 36.9)","1,358\n(947, 1,899)","35.8\n(9.0, 68.7)","695\n(174, 1,335)"
161,Bangladesh,"8.4\n(5.3, 12.8)","1,156\n(727, 1,765)","8.2\n(4.5, 13.3)","59\n(32, 97)"
201,Burkina Faso,"35.4\n(26.0, 45.7)","1,433\n(1,053, 1,848)","67.6\n(17.2, 144.7)","1,671\n(424, 3,576)"
202,Cameroon,"27.9\n(19.0, 38.5)","1,174\n(798, 1,619)","63.6\n(11.7, 137.4)","1,511\n(279, 3,265)"
6,China,"3.9\n(2.4, 6.1)","3,141\n(1,924, 4,995)","12.5\n(7.5, 18.8)","15\n(9, 23)"
205,Côte d'Ivoire,"26.2\n(18.3, 36.8)","1,051\n(734, 1,475)","19.8\n(7.2, 42.7)","493\n(179, 1,063)"
171,Democratic Republic of the Congo,"51.4\n(40.6, 62.5)","7,010\n(5,528, 8,513)","176.1\n(50.2, 390.1)","1,292\n(369, 2,862)"
141,Egypt,"3.0\n(1.8, 4.6)","320\n(198, 494)","3.9\n(1.5, 7.4)","36\n(14, 69)"
179,Ethiopia,"35.7\n(25.8, 47.4)","5,954\n(4,307, 7,906)","152.0\n(44.8, 290.9)","911\n(269, 1,744)"


In [149]:
pgd.format_data_summary(ntd_summary,
                        prevalence_multiplier=1000,
                        prevalence_decimals=2,
                        prevalent_cases_units=1,
                        daly_count_units=1000,
                        daly_count_decimals=1,
                        save_filename='data/neural_tube_defects_data_formatted.csv')

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,,Prevalence of Neural tube defects in age group Birth (per 1000),Number in age group Birth with Neural tube defects,DALYs attributable to Neural tube defects in age group Under 5 (1000s),"DALYs attributable to Neural tube defects per 100,000 person-years in age group Under 5"
location_id,location_name,,,,
168,Angola,"1.82\n(1.42, 2.29)","2,009\n(1,565, 2,531)","101.7\n(49.9, 180.7)","1,975\n(970, 3,511)"
161,Bangladesh,"0.82\n(0.64, 1.02)","2,169\n(1,710, 2,711)","105.0\n(40.2, 225.9)","764\n(293, 1,643)"
201,Burkina Faso,"2.47\n(1.91, 3.15)","2,315\n(1,794, 2,953)","215.0\n(81.9, 455.7)","5,316\n(2,024, 11,267)"
202,Cameroon,"1.83\n(1.43, 2.40)","1,657\n(1,294, 2,172)","100.9\n(46.5, 197.5)","2,397\n(1,106, 4,695)"
6,China,"0.26\n(0.21, 0.33)","3,936\n(3,179, 4,856)","225.9\n(176.5, 294.0)","277\n(217, 361)"
205,Côte d'Ivoire,"2.21\n(1.76, 2.84)","1,973\n(1,574, 2,536)","106.9\n(52.4, 196.1)","2,664\n(1,305, 4,887)"
171,Democratic Republic of the Congo,"2.62\n(2.02, 3.33)","7,692\n(5,937, 9,788)","285.1\n(142.4, 512.8)","2,092\n(1,045, 3,762)"
141,Egypt,"0.86\n(0.69, 1.07)","1,808\n(1,450, 2,251)","30.2\n(9.0, 80.6)","279\n(83, 744)"
179,Ethiopia,"1.12\n(0.87, 1.43)","4,120\n(3,189, 5,268)","257.8\n(147.6, 405.6)","1,545\n(885, 2,432)"


In [140]:
pd.Series(['a\nb'])

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    a\nb
dtype: object

In [142]:
print('a\nb')

a
b


In [ ]:
zinc_summary.to_csv()